In [ ]:
%%time

feature_MAE = []

#################################### Baseline ####################################
# define original feature lists
original_features = ['itemID', 'reviewerID', 'outOf_feature']

X_train_temp = X_train_filtered[original_features]
# X_val_temp = X_val_1[original_features]

regr = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0, n_jobs=-1)
regr.fit(X_train_temp, y_train_filtered)

y_pred = regr.predict(X_val[original_features])*np.array([i['outOf'] for i in X_val_raw['helpful']])
y_pred_train = regr.predict(X_train[original_features])*np.array([i['outOf'] for i in X_train_raw['helpful']])

# round to the nearest integer
y_pred = [int(round(i)) for i in y_pred]
y_pred_train = [int(round(i)) for i in y_pred_train]

print('Baseline Features')
print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
baseline_mae = mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)
feature_MAE.append(('baseline',baseline_mae))
print('MAE for val data: {}'.format(baseline_mae))
print('\n')

#################################### New Features ####################################
# define new feature lists
new_feature_list = ['rating', 'unixReviewTime', 'price',
                    'categoryID','categories_count', 'category_numtrans','summary_count_words', 
                    'reviewText_count_words', 'itemID_helpfulRate', 'reviewerID_helpfulRate',
                    'reviewerID_numReviews', 'rating_deviation', 'reviewText_posWordCount', 
                    'reviewText_negWordCount','reviewText_posWordRate', 'reviewText_negWordRate', 
                    'summary_count_char', 'reviewText_count_char', 'reviewText_count_punctu',
                    'summary_count_punctu','summary_posWordCount', 
                    'summary_negWordCount', 'summary_posWordRate','summary_negWordRate', 
                    'reviewText_count_firstCapital', 'summary_count_firstCapital',
                    'reviewText_avgWordLength','summary_avgWordLength', 'unixReviewTime_delta_firstreview',
                    'summary_reviewText_charRatio', 'summary_reviewText_wordsRatio',
                    'votes_time', 'itemID_numReviews', 'reviewText_capitalwords', 'summary_capitalwords',
                    'reviewText_ExclQue_countchar', 'summary_ExclQue_countchar',
                    'reviewText_PunctChar_ratio', 'summary_PunctChar_ratio', 'unixReviewTime_delta_lastreview']

features_tokeep = []
for feature in new_feature_list:
    
    X_train_temp = X_train_filtered[original_features + [feature]]
#     X_val = X_val_1[original_features + [feature]]
    
    regr = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0, n_jobs=-1)
    regr.fit(X_train_temp, y_train_filtered)

    y_pred = regr.predict(X_val[original_features + [feature]])*np.array([i['outOf'] for i in X_val_raw['helpful']])
    y_pred_train = regr.predict(X_train[original_features + [feature]])*np.array([i['outOf'] for i in X_train_raw['helpful']])

    # round to the nearest integer
    y_pred = [int(round(i)) for i in y_pred]
    y_pred_train = [int(round(i)) for i in y_pred_train]

    print(feature)
    
    print('Accuracy for train data: {}'.format(accuracy_score([i['nHelpful'] for i in y_train_raw],y_pred_train)))
    print('Accuracy for val data: {}'.format(accuracy_score([i['nHelpful'] for i in y_val_raw],y_pred)))
    
    print('MAE for train data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_train_raw], y_pred_train)))
    feature_mae = mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)
    feature_MAE.append((feature,feature_mae))
    print('MAE for val data: {}'.format(mean_absolute_error([i['nHelpful'] for i in y_val_raw], y_pred)))
    print('Difference from baseline: {}'.format(baseline_mae-feature_mae))
    print('\n')
    
    # keep features with lower MAE than baseline
    if feature_mae < baseline_mae:
        features_tokeep.append(feature)